# 读入数据

In [1]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from sklearn.decomposition import PCA
from numba import jit
import numba
import warnings
warnings.filterwarnings('ignore')

alphas = pd.read_csv("data_cutnorm.csv")
# alphas = pd.read_csv("data_residbarrarsector.csv")
base_data = pd.read_csv("base_data.csv")
# barra = pd.read_csv("barrar.csv")
barra = pd.read_hdf('barrar_risk.h5')

In [2]:
# 若添加barra因子加入组合，需合并
alphas = pd.merge(alphas, barra, on=['date', 'cn_code'], how='inner').dropna().reset_index().drop(columns='index')

# 加权方法
1. **等权法**：所有因子等权重相加并重新进行标准化处理，对应`AlphasCombination._weighted_average`方法。

2. **历史因子收益率（半衰）加权法**：计算因子历史收益率的算数平均值或半衰加权的历史平均收益率，作为因子加权的权重，对应`AlphasCombination._factor_return_weighting`方法。本例中使用默认半衰加权的方式，加权参数见后说明。

3. **历史因子IC（半衰）加权法**：计算因子历史RankIC的算数平均值或半衰加权的历史平均RankIC，作为因子加权的权重，对应对应`AlphasCombination._history_ic_weighting`方法。本例中使用默认半衰加权的方式，加权参数见后说明。；

4. **最大化ICIR加权法**：ICIR等于IC的期望值除以标准差。以历史因子平均IC值作为复合因子下一期IC的估计，以历史IC值的协方差矩阵作为对复合因子下一期波动率的估计，记$\bm{w}^*=(w_1^*,w_2^*,\dots,w_N^*)$为因子合成的权重，$\bm{IC}=(\overline{IC}_1, \overline{IC}_2, \dots,\overline{IC}_N)^T$表示因子IC的均值向量，$\Sigma$为因子IC的协方差矩阵，则问题变为 

$$\max ICIR=\frac{\bm{w}^T \bm{IC}}{\sqrt{\bm{w}^T \Sigma \bm{w}}}$$ 
显式解为$\bm{w}=\Sigma^{-1} \cdot \bm{IC}$，此后进行归一化处理。实际操作中，需要添加约束条件$\bm{w} \geq \bm{0}$，从而上述问题变为
$$\max ICIR=\frac{\bm{w}^T \bm{IC}}{\sqrt{\bm{w}^T \Sigma \bm{w}}}$$
$$\text{s.t. }\quad \bm{w}\geq 0$$ 

代码中，对应`AlphasCombination._max_icir_weighting`方法，加权权重由`_get_max_icir_weight_np`函数给出。此外，采用Ledoit & Wolf (2004)给出的压缩估计方法改善对真实协方差矩阵的估计，具体求解参考下方备注，对应`_shrinkage_covariance`函数。

5. **最大化IC加权法**：同样解优化问题
$$\max IC=\frac{\bm{w}^T \bm{IC}}{\sqrt{\bm{w}^T V \bm{w}}}$$
$$\text{s.t. }\quad \bm{w}\geq 0$$ 
其中相比最大化ICIR的优化方式，仅仅将$\Sigma$替换成了$V$，即当前截面因子值的相关系数矩阵。在代码中，对应此外，`AlphasCombination._max_ic_weighting`方法，加权权重由`_get_max_ic_weight_np`函数给出。我们同样对样本因子值的协方差矩阵做上述压缩估计以获得无偏估计。

6. **主成分分析估计法**：该方式直接对所有因子进行主成分分析降维，取第一主成分作为组合后的因子值。该方式的优势是组合结果通常较为稳定，不依赖于因子收益、IC/ICIR，但与此同时也可能因未充分利用历史信息而“欠拟合”。本研究中统一取总成分数为5的降维结果的第一主成分，对应`AlphasCombination._pca_weighting`方法。

## 备注
- **注1：半衰加权公式**：设半衰期参数为$H$，过去共有$T$期数据，加权权重$w=(w_1,w_2,\dots,w_t)$, $w_1$是距离现在最远的一期权重。则有初始$w_t$

$$w_t=2^{\frac{t-T-1}{H}}, \quad t=1,2,\dots,T$$
在实际操作中，需对上述权重进行归一化：
$$w_t'=\frac{w_t}{\sum w_t}$$
此方法可应用到历史IC加权组合，历史因子收益率加权组合等方式中，以提升近期因子表现的权重。本研究中，统一取$T=250,H=50$，即250个交易日历史数据（约1年）进行加权，权重每50个交易日变为原先的一半。

- **注2**：为使得加权结果稳定，在\textbf{因子组合模块}中使用的所有IC均为RankIC而非常规的PearsonIC

- **注3：协方差矩阵的压缩估计。**
设$\Sigma^*$是对真实协方差矩阵$\Sigma$在有限样本下的渐进一致估计，$S$是样本$X$的协方差矩阵，则需要找到$\Sigma^*$，使得$$\mathbb{E}\left[||\Sigma^*-\Sigma||^2\right]$$尽可能小，其中$||\cdot||$是矩阵的Frobenius范数（即所有元素的均方根），也即$$||A||=\sqrt{tr(AA^T)/N}$$ Ledoit和Wolf证明了对于样本$X=(x_t)$和其协方差矩阵$S(N\times T)$，该问题的压缩估计为 $$\Sigma^*=\rho_1 I+\rho_2 S$$ 其中 $$\rho_1=\frac{b^2}{d^2}m, \quad \rho_2=\frac{a^2}{d^2}$$ $$m=||S-I||^2, \quad d^2=||S-mI||^2, \quad b_0^2=\frac{1}{T^2}\sum_{t=1}^T ||x_t \cdot x_t^T-S||^2$$ $$b^2=\min(b_0^2,d^2), \quad a^2=d^2-b^2$$
此方法应用在因子优化中，最大化IC和ICIR的组合方式的协方差矩阵估计中。

In [3]:
@jit
def squared_fro(m):
    '''
    tr(AA^T)/n
    '''
    return np.trace(m.dot(m.T)) / m.shape[1]

@jit
def sum_squared_fro(x, s):
    '''
    sum_{n} ||xx^T-s||
    '''
    return squared_fro(x.reshape(-1, 1).dot(x.reshape(1, -1)) - s)

@jit
def _shrinkage_covariance(y: pd.DataFrame):
    '''
    Ledoit & Wolf (2004), Shrinkage of Covariance Matrix
    '''
    n_alphas = y.shape[1]
    n_obs = y.shape[0]
    sample_corr = pd.DataFrame(y).corr().values

    m0 = sample_corr - np.identity(n_alphas)
    m = squared_fro(m0)
    d0 = sample_corr - m * np.identity(n_alphas)
    d_squared = squared_fro(d0)
    b0_squared = np.nansum(np.apply_along_axis(\
        sum_squared_fro, axis=1, arr=y, s=sample_corr)) / np.power(n_obs, 2)

    b_squared = min(b0_squared, d_squared)
    a_squared = d_squared - b_squared
    rho1 = b_squared / d_squared * m
    rho2 = a_squared / d_squared
    sigma = rho1 * np.identity(n_alphas) + rho2 * sample_corr
    return sigma

@jit
def _get_max_icir_weight_np(rankic: pd.DataFrame):
    '''
    Max ICIR weight, Sigma^{-1} times IC.mean
    '''
    sigma = _shrinkage_covariance(rankic)
    weight_max_icir = np.linalg.inv(sigma).dot(rankic.mean(axis=0))
    weight_max_icir = weight_max_icir / weight_max_icir.sum()
    return weight_max_icir

@jit
def _get_max_ic_weight_np(v: pd.DataFrame):
    '''
    Max IC weight, V^{-1} times IC.mean
    '''
    if('date' in v.columns):
        v.drop(columns='date', inplace=True)
    sigma = _shrinkage_covariance(v)
    weight_max_ic = np.linalg.inv(sigma).dot(v.mean(axis=0))
    weight_max_ic = weight_max_ic / weight_max_ic.sum()

    return pd.Series(weight_max_ic, index=v.columns)

class AlphasCombination:
    def __init__(self, alphas: pd.DataFrame, base_data: pd.DataFrame, \
        test_days=1, T=250, H=50, rankic=None):

        # 保存alphas, base_data和标签
        self.alphas = alphas
        self.base_data = base_data
        self.label_cols = ['date', 'cn_code']
        self.alpha_cols = self.alphas.columns.drop(self.label_cols)
        self.T = T
        self.H = H
        self.rankic = rankic
        
        # 初始化数据：合并，取出收益目标列
        self._init_data(test_days)
    
    def _init_data(self, test_days=1):
        '''
        初始化数据：合并，取出收益目标列
        '''
        self.test_days = test_days
        # 因变量选定为未来n日收益，由test_days指定
        self.ylabel_col = self._get_ylabel_col(test_days)
        # 删除未来目标收益（因变量）中为NaN的观测
        self.data = pd.merge(alphas, base_data, on=['date', 'cn_code'], \
            how='left')

    @staticmethod
    def _get_ylabel_col(test_days=1):
        '''
        指定因变量的未来收益周期数
        '''
        if(test_days == 1):
            return 'adj_ret_p1'
        elif(test_days == 5):
            return 'adj_ret_p1_p5'
        elif(test_days == 10):
            return 'adj_ret_p1_p10'
        elif(test_days == 20):
            return 'adj_ret_p1_p20'
        else:
            raise ValueError('Test periods should be in [1, 5, 10, 20].')
            
    @property
    def halflife_weight(self):
        '''
        半衰加权，默认T为250交易日，H为50
        '''
        return np.power(2, [(t - self.T - 1) / self.H for t in range(1, self.T + 1)])
    
    def _history_averaging(self, x, method='decay'):
        '''
        历史数据（因子收益/IC）加权平均
        method仅当设置为decay时为半衰加权，否则为等权平均
        '''
        if(method == 'decay'):
            assert len(x) == len(self.halflife_weight)
            return np.nansum(x * self.halflife_weight) / np.sum(self.halflife_weight)
        return np.mean(x, axis=0)
    
    def _weighted_average(self, x: pd.DataFrame, weight: pd.DataFrame):
        '''
        给定权重后进行加权平均
        '''
        try:
            avg = (x[weight.columns] * weight.loc[x['date']].values).sum(axis=1)
            return avg
        except:
            # 数据缺失，raise warning，返回全0值
            print("Warning: No weight data for date " + str(x['date'].values[0]))
            return (x[weight.columns] * 0).sum(axis=1)
    
    def _get_wls_res(self, sub_data, value='params', test_days=1):
        '''
        Get WLS results
        未来某时间段内收益率对中性化后的因子暴露进行WLS，得到系数为因子回报率
        '''
        # 回归前删除部分column，如日期和代码
        x_cols = self.alphas.columns.drop(self.label_cols)
        # 测试收益日数，默认为1
        y_col = self._get_ylabel_col(test_days)
        # 进行WLS回归，市值平方根加权
        res = sm.WLS(sub_data[y_col], sm.add_constant(sub_data[x_cols]), \
            weights=np.sqrt(sub_data['cap']), missing='drop').fit()
        # 返回需要的结果
        if(value == 'params'):
            return res.params
        elif(value == 'tvalues'):
            return res.tvalues
        elif(value == 'pvalues'):
            return res.pvalues
        else:
            raise ValueError("Unsupported value")

    def _result_wrapping(self, results, name):
        """
        Wrap the results series with date and code
        """
        assert isinstance(results, pd.Series)
        results_ = self.alphas[[self.label_cols]].reset_index().drop(
            columns='index')
        results_[name] = results.values
        return results_
    
    def _average_weighting(self):
        """
        Simple average of cross-sectional alphas
        """
        if(self.rankic is None):
            self._init_rankic(self.test_days)
        sign = (self.rankic.mean() < 0) * (-1) + (self.rankic.mean() >= 0) * 1
        self.average_alpha = (self.alphas.drop(columns=self.label_cols) * sign).mean(axis=1)
        return self.average_alpha
    
    def _factor_return_weighting(self, test_days=1):

        if(test_days != self.test_days):
            self._init_data(test_days)
        
        # WLS回归系数，为因子收益率
        data_dropna = self.data.dropna(subset=[self.ylabel_col])
        wls_params = data_dropna.groupby('date').apply(self._get_wls_res, \
            value='params', test_days=test_days)
        # 求过去T个交易日的滚动因子收益，按H个交易日作为半衰期做衰减加权
        # 默认T为250，H为50，注意去除常数项
        # NOTE: 权重向后平移Test Days，防止look-ahead bias
        weight_ret = wls_params.drop(columns = 'const').rolling(self.T).apply(
            self._history_averaging).shift(self.test_days)
        # 对加权权重进行归一化，得到长度为T*M的每日因子权重数据
        weight_ret = weight_ret.apply(lambda x: x / np.nansum(x), axis=1)
        # 对每日的所有股票的横截面数据，每支股票的因子暴露均按当日的权重加权求和进行组合
        self.ret_weighted_alpha = self.alphas.groupby('date').apply(\
            self._weighted_average, weight=weight_ret)
        return self.ret_weighted_alpha
    
    def _init_rankic(self, test_days=1):

        print("Initializing RankIC for Alphas now------------")
        if(test_days != self.test_days):
            self._init_data(test_days)
        
        # 因子值排序
        rank = self.alphas.drop(columns=['cn_code']).groupby('date').apply(\
            lambda x: x.rank(pct=True)).drop(columns='date')
        # 添加日期
        rank.insert(0, 'date', self.alphas['date'].values)
        # 因变量（未来期限收益）排序
        rank['rank_rtn'] = self.data.groupby('date')[\
            self.ylabel_col].rank(pct=True).values

        # 计算rankic
        self.rankic = rank.groupby('date').apply(\
            lambda x: x.corrwith(x['rank_rtn'])).drop(columns=['date', 'rank_rtn'])
        print("RankIC Initialized!---------------------------")
        return rank

    def _history_ic_weighting(self, test_days=1):

        if(test_days != self.test_days):
            self._init_data(test_days)
        if(self.rankic is None):
            self._init_rankic(test_days)
        
        # NOTE: 权重向后平移Test Days，防止look-ahead bias
        weight_ic = self.rankic.rolling(self.T).apply(self._history_averaging).shift(self.test_days)
        weight_ic = weight_ic.apply(lambda x: x / np.nansum(x), axis=1)
        self.ic_weighted_alpha = self.alphas.groupby('date').apply(\
            self._weighted_average, weight=weight_ic)
        return self.ic_weighted_alpha
    
    def _pca_weighting(self, n_components=5):
        '''
        对因子进行PCA变换，取第一主成分作为组合后的因子，默认主成分数为5
        '''
        pca = PCA(n_components=n_components)
        first_comp = pca.fit_transform(self.alphas.drop(columns=self.label_cols))[:, 0]
        self.pca_weighted_alpha = pd.Series(first_comp)
        return self.pca_weighted_alpha
    
    def _max_icir_weighting(self, test_days=1):
        '''
        对因子进行最大化ICIR加权，公式见说明文档
        '''
        if(test_days != self.test_days):
            self._init_data(test_days)
        if(self.rankic is None):
            self._init_rankic(test_days=test_days)
        
        # 对所有数据滚动T期按公式计算权重
        # 注意：pd的rolling操作不支持在table上进行滚动，使用numba引擎保证计算可以实现
        # NOTE: 权重向后平移Test Days，防止look-ahead bias
        weight_max_icir = self.rankic.rolling(self.T, min_periods=self.T, \
            method='table').apply(_get_max_icir_weight_np, engine='numba', \
                raw=True, engine_kwargs={'nopython': False}).shift(self.test_days)
        
        # 加权平均
        self.max_icir_weighted_alpha = self.alphas.groupby('date').apply(\
            self._weighted_average, weight=weight_max_icir)
        return self.max_icir_weighted_alpha
    
    def _max_ic_weighting(self, test_days=1):
        '''
        对因子进行最大化IC加权，公式见说明文档
        '''
        if(test_days != self.test_days):
            self._init_data(test_days)
        
        # 对每日的因子值截面相关性系数矩阵按公式计算权重
        weight_max_ic = self.alphas.drop(columns=['cn_code']).groupby(\
            'date').apply(_get_max_ic_weight_np)
        # 加权平均
        self.max_ic_weighted_alpha = self.alphas.groupby('date').apply(\
            self._weighted_average, weight=weight_max_ic)
        return self.max_ic_weighted_alpha
    
    def fit(self):
        '''
        汇总运行各个方法并保存结果，API接口
        '''

        print("Aggregation begins for future {:d} days of return".format(self.test_days))
        
        print("Performing Average Weighting------------------")
        self._average_weighting()
        print("Average Rating Finished!----------------------\n")

        print("Performing PCA Weighting----------------------")
        self._pca_weighting()
        print("PCA Rating Finished!--------------------------\n")

        print("Performing History Factor Return Weighting----")
        self._factor_return_weighting(self.test_days)
        print("Factor Return Weighting Finished!-------------\n")

        print("Performing History IC Weighting---------------")
        self._history_ic_weighting(self.test_days)
        print("History IC Weighting Finished!----------------\n")

        print("Performing Maximizing ICIR Weighting----------")
        self._max_icir_weighting(self.test_days)
        print("Maximizing ICIR Weighting Finished!-----------\n")

        print("Performing Maximizing IC Weighting------------")
        self._max_ic_weighting(self.test_days)
        print("Maximizing IC Weighting Finished!-------------\n")

        print("Aggregation Finished! Organizing results now!")

        # 保存结果
        self.alpha_aggregations = pd.DataFrame(\
            self.average_alpha.values, index = self.ret_weighted_alpha.index, \
            columns=['Average'])
        self.alpha_aggregations.insert(0, 'cn_code', self.alphas['cn_code'].values)
        self.alpha_aggregations['PCA'] = self.pca_weighted_alpha.values
        self.alpha_aggregations['HistoryFactorReturn'] = self.ret_weighted_alpha.values
        self.alpha_aggregations['HistoryIC'] = self.ic_weighted_alpha.values
        self.alpha_aggregations['MaxICIR'] = self.max_icir_weighted_alpha.values
        self.alpha_aggregations['MaxIC'] = self.max_ic_weighted_alpha.values
        print("Result Organization Finished!-----------------\n")

        print("Saving Your Results Now!----------------------")
        save_path = "alpha_aggregations_" + self.ylabel_col + ".h5"
        self.alpha_aggregations.to_hdf(save_path, key='stage', mode='w')
        print("Aggregation Results Saved! ^_^ ---------------")

        return self.alpha_aggregations

In [36]:
# average_generator = AlphasCombination(alphas, base_data, test_days=1)
# average_generator._average_weighting()
# avg_alpha = pd.DataFrame(average_generator.average_alpha, columns=['average'])
# avg_alpha.insert(0, 'date', alphas.date)
# avg_alpha.insert(1, 'cn_code', alphas.cn_code)
# avg_alpha.to_hdf("alpha_aggregations_average_original.h5", key='stage', mode='w', complevel=9)

Initializing RankIC for Alphas now------------
RankIC Initialized!---------------------------


0          0.366047
1         -0.274629
2         -0.316559
3         -0.084904
4         -0.092602
             ...   
4404950   -0.223424
4404951   -0.247717
4404952   -0.261617
4404953   -0.015019
4404954    0.378353
Length: 4404955, dtype: float64

In [4]:
for test_days in [1, 5, 10, 20]:
    generator = AlphasCombination(alphas, base_data, test_days=test_days)
    generator.fit()

Aggregation begins for future 1 days of return
Performing Average Weighting------------------
Initializing RankIC for Alphas now------------
RankIC Initialized!---------------------------
Average Rating Finished!----------------------

Performing PCA Weighting----------------------
PCA Rating Finished!--------------------------

Performing History Factor Return Weighting----
Factor Return Weighting Finished!-------------

Performing History IC Weighting---------------
History IC Weighting Finished!----------------

Performing Maximizing ICIR Weighting----------
Maximizing ICIR Weighting Finished!-----------

Performing Maximizing IC Weighting------------
Maximizing IC Weighting Finished!-------------

Aggregation Finished! Organizing results now!
Result Organization Finished!-----------------

Saving Your Results Now!----------------------
Aggregation Results Saved! ^_^ ---------------
Aggregation begins for future 5 days of return
Performing Average Weighting------------------
Initial